## Introduction

In [306]:
### Functions needed to import and structure Crunchbase data
# Written by David Chataway

In [307]:
# Import libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
import re
from dateutil.relativedelta import relativedelta
import math
from scipy.signal import find_peaks, peak_widths
import seaborn as sns

## Crunchbase

In [308]:
def read_crunchbase_export(filename, queries):
    # Filename is a string including the .csv 
    # queries is a dictionary of the column:filter
    
    ##################################################
    # READ 
    ##################################################
    # Read csv, then structure and prepare it based on the query provided
    df = pd.read_csv("Crunchbase/exports/"+filename)

    ##################################################
    # CONFIGURE AND QUERY 
    ##################################################
    # Date configuration - extract date of date of data extraction 
    match = re.findall(r'[0-9]+-[0-9]+-[0-9]{4}', filename)
    format = "%m-%d-%Y"
    dt_object = datetime.strptime(match[0], format)

    # Convert into a date range with T-5 years
    start_date = dt_object - relativedelta(years=5)

    # Create date range for later
    dates = pd.date_range(start=start_date,end=dt_object)
    df_day = pd.DataFrame({'date':dates})

    # Query and filter the dataframe
    df_queried = df.copy(deep=True)
    df_queried['Announced Date'] = pd.to_datetime(df_queried['Announced Date'])
    if len(queries) == 0:
        pass
    else:
        for key in queries.keys():
            df_queried = df_queried[df_queried[key] == queries[key]]
    
    ##################################################
    # TRANSFORM TO DATE 
    ##################################################    
    
    # USER DEFINED COLUMNS TO AGGREGATE
    y_cols = {'Total Funding Amount Currency (in USD)':[], 'Money Raised Currency (in USD)':[], 'Pre-Money Valuation Currency (in USD)':[]}
    # Count number of founding rounds
    ls_count = []

    # Generate aggregated dataframe by day
    cols = list(y_cols.keys())
    cols.append('Announced Date')
    df_grouped = df_queried[cols].groupby('Announced Date', as_index=False).sum()

    # Loop once through dates
    for i in dates:
        count = 0
        # Count the entries
        for j in df_queried['Announced Date']:
            if (j >= i and j < (i+timedelta(days=1))):
                count += 1
        ls_count.append(count)

        # Get aggregate sums by date
        if df_grouped.loc[df_grouped['Announced Date'] == i].empty:
            for key in y_cols.keys():
                y_cols[key].append(0)
        else:
            for key in y_cols.keys():
                y_cols[key].append(float(df_grouped[key][df_grouped['Announced Date'] == i]))

    # Add the information to the df_day dataframe  
    df_day['count'] = ls_count
    for key in y_cols.keys():
        df_day[key] = y_cols[key]

    # GROUPING BY WEEK
    #df_day['Start_of_week'] = pd.to_datetime(df_day['date']).to_period('W').start_time
    df_week = df_day.groupby(pd.Grouper(key='date', axis=0, 
                          freq='W')).sum().reset_index()
    #df_week['week'] =  df_week['date'].dt.week
    #df_week['year'] =  df_week['date'].dt.year
    
    # Returning the Start of the week for each date
    ls_start_of_week = []
    for date in df_week['date']:
        ls_start_of_week.append(date.to_period('W').start_time)
    df_week['Start_of_week'] = ls_start_of_week
        
    return df, df_queried, df_day, df_week

In [309]:
## Test
filename = "web-3-search-3-11-2023.csv"
queries = {
    'Equity Only Funding':'Yes'
    #,'Funding Type':'Seed'
}
df_raw, df_queried, df_day, df_week = read_crunchbase_export(filename, queries)
df_week

,date,count,Total Funding Amount Currency (in USD),Money Raised Currency (in USD),Pre-Money Valuation Currency (in USD),Start_of_week
0,2018-03-11,0,0.0,0.0,0.0,2018-03-05
1,2018-03-18,0,0.0,0.0,0.0,2018-03-12
2,2018-03-25,1,15000000.0,15000000.0,0.0,2018-03-19
3,2018-04-01,1,4480000.0,1230000.0,0.0,2018-03-26
4,2018-04-08,0,0.0,0.0,0.0,2018-04-02
...,...,...,...,...,...,...
257,2023-02-12,1,4900000.0,4500000.0,0.0,2023-02-06
258,2023-02-19,4,19075000.0,19075000.0,0.0,2023-02-13
259,2023-02-26,5,14600000.0,12800000.0,13000000.0,2023-02-20
260,2023-03-05,3,7400000.0,7400000.0,0.0,2023-02-27


## Google Trends

In [310]:
def read_google_trends_export(filename_google):
    # Filename is a string including the .csv 

    ##################################################
    # READ 
    ##################################################
    # read csv and extract metadata of the search contained in the first cell
    df_google = pd.read_csv("Google_Trends/exports/"+filename_google)
    
    if df_google.iloc[1][0] == 'Week':
        Google_search_meta = df_google.iloc[1]
        df_google = df_google.drop([0], axis = 0)
    else:
        Google_search_meta = df_google.iloc[0]

    ##################################################
    # CONFIGURE 
    ##################################################

    # Structure the dataframe
    try:
        df_google = df_google.drop(['Week'], axis = 0)
    except:
        df_google = df_google.drop([1], axis = 0)
        df_google = df_google.set_index('Category: All categories')
        
    # Cast the Week range as a datetime64 type
    # df_google['date'] = df_google.index.astype('datetime64[ns]')
    #df_google['week'] =  pd.to_datetime(df_google.index).week
    #df_google['year'] =  pd.to_datetime(df_google.index).year
    df_google['Start_of_week'] = pd.to_datetime(df_google.index).to_period('W').start_time

    return df_google #, Google_search_meta

In [311]:
## Test
filename_google = "Web3-Google-3-11-2023.csv"
df_google = read_google_trends_export(filename_google)
df_google

,Category: All categories,Start_of_week
2018-03-18,5,2018-03-12
2018-03-25,5,2018-03-19
2018-04-01,5,2018-03-26
2018-04-08,4,2018-04-02
2018-04-15,4,2018-04-09
...,...,...
2023-02-05,40,2023-01-30
2023-02-12,36,2023-02-06
2023-02-19,37,2023-02-13
2023-02-26,43,2023-02-20


## Twitter

In [312]:
# TO BE COMPLETED
# PULL HASHTAGS OF TOP VCs

# For instance, Sequoia Capital

## Merging Crunchbase and Google Datasets
##### By week

In [313]:
def merge_weekly(df_google, df_crunchbase):

    # Requires two dataframes, where
    # [0] = Google df
    # [1] = Crunchbase df
        
    df_combined_week = pd.DataFrame()
    #df_combined_week = df_google.merge(df_crunchbase, on = ['week','year'], how = 'inner')
    df_combined_week = df_google.merge(df_crunchbase, on = ['Start_of_week'], how = 'inner')
    
    return df_combined_week

In [314]:
## Test
df_combined_week = merge_weekly(df_google, df_week)
df_combined_week

,Category: All categories,Start_of_week,date,count,Total Funding Amount Currency (in USD),Money Raised Currency (in USD),Pre-Money Valuation Currency (in USD)
0,5,2018-03-12,2018-03-18,0,0.0,0.0,0.0
1,5,2018-03-19,2018-03-25,1,15000000.0,15000000.0,0.0
2,5,2018-03-26,2018-04-01,1,4480000.0,1230000.0,0.0
3,4,2018-04-02,2018-04-08,0,0.0,0.0,0.0
4,4,2018-04-09,2018-04-15,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
255,40,2023-01-30,2023-02-05,3,36100000.0,24000000.0,0.0
256,36,2023-02-06,2023-02-12,1,4900000.0,4500000.0,0.0
257,37,2023-02-13,2023-02-19,4,19075000.0,19075000.0,0.0
258,43,2023-02-20,2023-02-26,5,14600000.0,12800000.0,13000000.0


## Post Processing

In [315]:
def post_processing(dfs, window_size = 3):
    dfs_return = {}
    # Requires dictionary of dfs
    # window is an int that describes the size of the moving average window
    
    ##################################################
    # STANDARDIZE COLUMNS TO 100
    ##################################################  
    
    # Make $ columns relative
    y_cols = {'Total Funding Amount Currency (in USD)':[], 'Money Raised Currency (in USD)':[], 'Pre-Money Valuation Currency (in USD)':[]}

    # Loop through the dataframes
    for key in dfs.keys(): 
        
        # Copy the dataframe
        df_return = dfs[key].copy(deep=True)

        post_process_cols = []

        for key_cols in y_cols.keys():
            col_name = key_cols+' %'
            post_process_cols.append(key_cols)
            post_process_cols.append(col_name)
            df_return[col_name] = df_return[key_cols] / max(df_return[key_cols]) * 100

        df_return['count %'] = df_return['count'] / max(df_return['count']) * 100
        post_process_cols.append('count')
        post_process_cols.append('count %')

        # Make Google Trends info a column
        try:
            df_return['Google Trends'] = df_return['Category: All categories']
        except:
            df_return['Google Trends'] = df_return['Unnamed: 1']
        # Correct for <1
        df_return.loc[df_return['Google Trends'] == '<1', 'Google Trends'] = 0
        # Cast as float
        df_return['Google Trends'] = df_return['Google Trends'].astype(float)
        post_process_cols.append('Google Trends')

        ##################################################
        # SMOOTH VIA MOVING AVERAGE 
        ##################################################  
        
        # simple moving averages using pandas; don't need a spline because we aren't looking to fit a smoother to the data
        for col in post_process_cols:
            col_name = col + '-Rolling'

            # Get the window of series ... # of observations of specified window size
            windows = df_return[col].rolling(window_size, center = True)

            # Create a series of moving averages of each window
            moving_averages = windows.mean()

            # Correct_null values
            if window_size == 3:
                moving_averages[0] = df_return[col][0]
                moving_averages[len(moving_averages)-1] = df_return[col][len(moving_averages)-1]
            elif window_size == 5: 
                moving_averages[0] = df_return[col][0]
                moving_averages[1] = df_return[col][1]
                moving_averages[len(moving_averages)-1] = df_return[col][len(moving_averages)-1]
                moving_averages[len(moving_averages)-2] = df_return[col][len(moving_averages)-2]
            else: 
                raise ValueError('Please only use window size of 3 or 5 or update the code')
                
            # Assign
            df_return[col_name] = moving_averages
            
            # Re-standardize based on smoothed columns (max = 100)
            if ' %' in col_name:
                pass
            else:
                df_return[str(col_name + ' %')] = df_return[col_name] / max(df_return[col_name]) * 100
        
        # Append to dictionary
        dfs_return[key] = df_return
        
    return dfs_return

## Identification of Peak Hype and Peak Funding

Method: The local maxima would be any x point which has a higher y value than either of its left and right neighbors. To eliminate noise, you could put in some kind of tolerance threshold (ex. x point must have higher y value than n of its neighbors). And specify a distance away threshold.

Other Methods for Peak Finding https://www.originlab.com/index.aspx?go=Products/Origin/DataAnalysis/PeakAnalysis#:~:text=Peak%20Finding%2FDetermination%201%20Savitzky-Golay%20smoothing%20on%20the%20spectrum,Derivative%20%28for%20hidden%20peaks%29%20Fourier%20Self%20Deconvolution%20PRO

1. Savitzky-Golay smoothing on the spectrum before peak finding
2. 6 methods for finding peaks: i) Local Maximum, ii) Window Search, iii) First Derivative, iv) Second Derivative (for hidden peaks), v) Residual after 1st Derivative (for hidden peaks), vi) Fourier Self Deconvolution PRO
3. After finding peaks, you can filter out unwanted peaks: i) By peak height, ii) By number of peaks, iii) Label peaks with X, Y values or row index

In [316]:
def peaks (dfs, min_height = 0.6, min_threshold = 0.00, min_spacing = 2, rolling = True):
    # Takes in the dictionary of dataframes and outputs a dictionary with the peaks
    # Min height is the necessary absolute height to be considered a peak
    # Threshold is necessary relative height difference (vs neighbors) to be considered a subsequent peak
    # Min spacing is the minimum time period (i.e. weeks) mandated between peaks
    
    dfs_peaks = {}

    # Loop through the dataframes
    for key in dfs.keys(): 
        peaks_dict = {}

        if rolling:
            cols = ['Google Trends-Rolling',
                    'Google Trends-Rolling %',
                    'count-Rolling', 'count-Rolling %', 'count %-Rolling', 
                    'Total Funding Amount Currency (in USD)-Rolling',
                    'Total Funding Amount Currency (in USD)-Rolling %',
                    'Total Funding Amount Currency (in USD) %-Rolling',
                    'Money Raised Currency (in USD)-Rolling',
                    'Money Raised Currency (in USD)-Rolling %',
                    'Money Raised Currency (in USD) %-Rolling',
                    'Pre-Money Valuation Currency (in USD)-Rolling',
                    'Pre-Money Valuation Currency (in USD)-Rolling %',
                    'Pre-Money Valuation Currency (in USD) %-Rolling']
        else:
            cols = ['Google Trends',
                    'count',
                    'Total Funding Amount Currency (in USD)', 
                    'Money Raised Currency (in USD)', 
                    'Pre-Money Valuation Currency (in USD)'
                    'Total Funding Amount Currency (in USD) %',
                    'Money Raised Currency (in USD) %',
                    'Pre-Money Valuation Currency (in USD) %', 
                    'count %']

        ##################################################
        # FIND PEAK HYPE (GOOGLE SEARCHES) 
        ################################################## 
        
        # Finds the peak of Google Trends (col no 0)
        peaks_dict['Hype'] = dfs[key]['date'].iloc[dfs[key][cols[0]].idxmax()]
        # Break ties with the earlier date
        if isinstance(peaks_dict['Hype'], list):
            peaks_dict['Hype'] = min(peaks_dict['Hype'])
        
        ##################################################
        # FIND PEAKS OF FUNDING 
        ##################################################        
        
        for col in cols:
            # https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.argrelextrema.html
            # https://stackoverflow.com/questions/35282456/find-local-maximums-in-numpy-array
            # Statistical methods vs analytical ones?
            
            # For each column, return array of peaks
            peaks, properties = find_peaks(dfs[key][col], distance= min_spacing, height=min_height*max(dfs[key][col]), threshold = min_threshold*max(dfs[key][col]))
            if len(peaks) == 0:
                peaks = np.array([0])
            peaks_dict[col + '_array'] = peaks
            peaks_dict[col + '_array_properties'] = properties
            
            # Get the complete width (~3 sigma) at peaks 
            full_widths = peak_widths(dfs[key][col], peaks, rel_height=1)
            peaks_dict[col + '_full_widths'] = full_widths
            peaks_dict[col + '_array_widths'] = full_widths[0] 
            
            # Find the earliest peak
            peaks_dict[col] = min(peaks)
            peaks_dict[col + '_date'] = dfs[key]['date'][min(peaks)]
            peaks_dict[col + '_width'] = full_widths[0][np.where(peaks==min(peaks))[0][0]] 
            
            # Find the highest peak
            peaks_dict[col + '_max'] = dfs[key][col].idxmax()
            # Break ties with the earlier date
            if isinstance(peaks_dict[col + '_max'], list):
                peaks_dict[col + '_max'] = min(peaks_dict[col + '_max'])
            elif math.isnan(peaks_dict[col + '_max']):
                peaks_dict[col + '_max'] = 0
            peaks_dict[col + '_max_date'] = dfs[key]['date'].iloc[peaks_dict[col + '_max']]
            
        # Assign to DataFrame
        dfs_peaks[key] = peaks_dict
        
    return dfs_peaks

## Recreate a Hype Cycle

In [317]:
#TBD

## Plotting

In [318]:
sns.set(font_scale=2)


In [319]:
def plot_time_single_technology (key, x_col, y_1_col, y_2_col, saving_plots, standardize):

    x = df_mains[key][x_col]
    y_smooth = df_mains[key][y_1_col]
    z_smooth = df_mains[key][y_2_col]
    if standardize:
        y_smooth = y_smooth * 100 / max(y_smooth)
        z_smooth = z_smooth * 100 / max(z_smooth)

    plt.figure(figsize=(24, 12))
    sns.set_style("whitegrid", {"grid.color": ".6", "grid.linestyle": ":"})

    plt.plot(x,y_smooth, color = 'g', label = y_1_col)

    plt.plot(x,z_smooth, color='b', label = y_2_col)

    plt.title(key + ' - Early Stage Funding and Google Trends by Week')
    plt.legend()
    plt.ylim([0,100])
    if saving_plots:
        plt.savefig("Plots/Time-Series/" + key + '_standardized='+str(standardize)+'.png', dpi = 1000)
    plt.show()

In [320]:
def plot_time_single_technology_peaks (key, x_col, y_1_col, y_2_col, saving_plots, standardize):

    x = df_mains[key][x_col]
    y_smooth = df_mains[key][y_1_col]
    z_smooth = df_mains[key][y_2_col]
    if standardize:
        y_smooth = y_smooth * 100 / max(y_smooth)
        z_smooth = z_smooth * 100 / max(z_smooth)

    plt.figure(figsize=(24, 12))
    sns.set_style("whitegrid", {"grid.color": ".6", "grid.linestyle": ":"})

    plt.plot(x,y_smooth, color = 'g', label = y_1_col)

    plt.plot(x,z_smooth, color='b', label = y_2_col)

    #PLOT PEAKS
    plt.axvline(x = df_peaks[key]['Hype'], label = 'Moment Peak Hype', color="C4")
    plt.axvline(x = df_peaks[key][y_2_col +'_date'], label = 'Earliest Moment of Peak Hype', color="C4", ls = '--')
    plt.axvline(x = df_peaks[key][y_1_col +'_date'], label = 'Earliest Moment of Peak Funding', color="black")

    peak_indices = df_peaks[key][y_1_col +'_array']
    plt.plot(x[peak_indices], y_smooth[peak_indices], "x", label = 'Peaks in Funding', color = 'black', markersize=16)

    plt.title(key + ' - Early Stage Funding and Google Trends by Week - with Peaks')
    plt.legend()
    plt.ylim([0,100])
    if saving_plots:
        plt.savefig("Plots/Time-Series/" + key + '_smooth_peaks.png', dpi = 1000)
    plt.show()

In [321]:
def plot_time_multi_technology_peaks (keys, x_col, y_1_col, y_2_col, saving_plots, standardize):
    
    df_return = pd.DataFrame(columns = ["Key", "Title", "Value"])
    
    plt.rcParams.update({'font.size': 16})
    sns.set_style("whitegrid", {"grid.color": ".6", "grid.linestyle": ":"})

    fig, axs = plt.subplots(nrows=len(keys), ncols=1, figsize=(24, 56))
    fig.suptitle('Early Stage Funding and Google Trends\nby Week (with Peaks shown)', fontsize=32)

    for i in range(0,len(keys)):
        key = keys[i]
        
        x = df_mains[key][x_col]
        y_smooth = df_mains[key][y_1_col]
        z_smooth = df_mains[key][y_2_col]
        if standardize:
            y_smooth = y_smooth * 100 / max(y_smooth)
            z_smooth = z_smooth * 100 / max(z_smooth)

        #Plot Google trends
        axs[i].plot(x,z_smooth, color='b', label = y_2_col)
        axs[i].axvline(x = df_peaks[key]['Hype'], label = 'Moment of Peak Hype', color="darkblue")
        axs[i].axvline(x = df_peaks[key][y_2_col +'_date'], label = 'Earliest Moment of Peak Hype', color="darkblue", ls = '--')
        axs[i].axvspan(df_peaks[key]['Hype'], df_peaks[key][y_2_col +'_date'], alpha=0.3, color='lightsteelblue', label = 'Hype Innovation Trigger Range')
    
        # Plot funding
        axs[i].plot(x,y_smooth, color = 'g', label = y_1_col)
        axs[i].axvline(x = df_peaks[key][y_1_col +'_date'], label = 'Earliest Moment of Peak Funding', color="black")
        axs[i].axvspan(df_peaks[key][y_1_col +'_date'], df_peaks[key][y_1_col +'_max_date'], alpha=0.3, color='palegreen', label = 'Innovation Funding Range')
        peak_indices = df_peaks[key][y_1_col +'_array']
        axs[i].plot(x[peak_indices], y_smooth[peak_indices], "x", label = 'Peaks in Funding', color = 'black', markersize=12)

        axs[i].set_title(key)
        #axs[i].legend()
        axs[i].set_ylim([0,100])
        
        # Add to return dataframe
        df_return = df_return.append({'Key': key, 'Title': 'Earliest Moment of Peak Hype', 'Value': df_peaks[key][y_2_col +'_date']}, ignore_index=True)
        df_return = df_return.append({'Key': key, 'Title': 'Moment of Peak Hype', 'Value': df_peaks[key]['Hype']}, ignore_index=True)
        df_return = df_return.append({'Key': key, 'Title': 'Earliest Moment of Peak Funding', 'Value': df_peaks[key][y_1_col +'_date']}, ignore_index=True)
        df_return = df_return.append({'Key': key, 'Title': 'Moment of Peak Funding', 'Value': df_peaks[key][y_1_col +'_max_date']}, ignore_index=True)
        ### KEY - TIME DELTA defined by EARLIEST FUNDING - EARLIEST HYPE moments
        df_return = df_return.append({'Key': key, 'Title': 'Time Delta (Days)', 'Value': (df_peaks[key][y_1_col +'_date'] - df_peaks[key][y_2_col +'_date']).days}, ignore_index=True)
        df_return = df_return.append({'Key': key, 'Title': 'Time Delta (Days) - Max Amounts', 'Value': (df_peaks[key][y_1_col +'_max_date']-df_peaks[key]['Hype']).days}, ignore_index=True)

    # Single legend
    handles, labels = axs[0].get_legend_handles_labels()
    fig.legend(handles, labels) #, loc='upper center')

    # Fix formatting
    fig.tight_layout()
    fig.subplots_adjust(top=0.93)

    if saving_plots:    
        plt.savefig("Plots/Time-Series/All_Technologies_with_Peaks.png", dpi = 300)
        df_return.to_csv('Data/time-series-results.csv')
    
    plt.show()
    return plt, fig, df_return

In [322]:
def plot_history_summary (keys, df_return):
    x = list(df_return[df_return['Title'] == 'Time Delta (Days)']['Value'] / 365)
    y = list(df_return[df_return['Title'] == 'Earliest Moment of Peak Hype']['Value'])

    sns.set(rc={'figure.figsize': (16, 8)})
    # plotting scatter plot
    sns.set_style("darkgrid")
    plt.scatter(x, y, label = 'Data') #sns.scatterplot(x, y)
    plt.ylim(max(y)+timedelta(days=180), min(y)-timedelta(days=180)) #show year in descending order

    # Mean and STD
    average = np.mean(df_return[df_return['Title'] == 'Time Delta (Days)']['Value']) 
    print('The average time difference between hype and funding cycles is: ' + str(average))
    st_dev = np.std(df_return[df_return['Title'] == 'Time Delta (Days)']['Value'])
    print('The standard deviation is: ' + str(st_dev))
    
    # Plot helper lines
    plt.axvline(x = (average/365), ls = '--', label = 'Average Funding Time Lag')
    plt.axvline(x = 0, ls = '-', color = 'black')
    plt.axvspan((average-st_dev)/365, (average+st_dev)/365 , alpha=0.2, color='lightsteelblue', label = 'Standard Deviation of the Mean')

    # Loop for annotation of all points
    for i in range(len(x)):
        plt.annotate(keys[i], (x[i]+0.02, y[i]))
    # annotation of the third point
    # plt.text(2,4.2,"third")    
    
    plt.ylabel('Date Corresponding to Start of Hype Cycle', fontsize = 16)
    plt.xlabel('Time Difference (Years) Between Start of Hype Cycle and Start of Funding Cycle', fontsize = 16)
    plt.title('Recent Innovations and the Early-Stage Funding Lag', fontsize = 20)
    plt.legend()
    
    if saving_plots:
        plt.savefig("Plots/Time-Series/Time_Difference_History_Summary.png")
    plt.show()
    

## Transform Dates Based on T- Hype

In [323]:
def relative_dates(keys, df_mains, df_return):
    # Make a new columns with dates relative to the hype cycle
    
    for key in keys:
        df_mains[key]['relative_date_days'] = df_mains[key]['date'] - (df_return[(df_return['Key']==key) & (df_return['Title']=='Earliest Moment of Peak Hype')]['Value']).iloc[0]
        df_mains[key]['relative_date_days'] = df_mains[key]['relative_date_days'].dt.days
        df_mains[key]['relative_date_weeks'] = df_mains[key]['relative_date_days']/7
        df_mains[key]['relative_date_months'] = df_mains[key]['relative_date_days']/30.5
        df_mains[key]['relative_date_years'] = df_mains[key]['relative_date_days']/365

        
    return df_mains

In [324]:
def relative_dates_df(key, df, df_return):
    # Make a new columns with dates relative to the hype cycle
    
    df['relative_date_days'] = df['Announced Date'] - (df_return[(df_return['Key']==key) & (df_return['Title']=='Earliest Moment of Peak Hype')]['Value']).iloc[0]
    df['relative_date_days'] = df['relative_date_days'].dt.days
    df['relative_date_weeks'] = df['relative_date_days']/7
    df['relative_date_months'] = df['relative_date_days']/30.5
    df['relative_date_years'] = df['relative_date_days']/365
        
    return df

In [325]:
def relative_dates_dfs(keys, dfs, df_return):
        
    # Make a new columns with dates relative to the hype cycle
    count = 0
    
    for key in keys:
        if count == 0:
            df = dfs[key][1].copy(deep=True)

            df['relative_date_days'] = df['Announced Date'] - (df_return[(df_return['Key']==key) & (df_return['Title']=='Earliest Moment of Peak Hype')]['Value']).iloc[0]
            df['relative_date_days'] = df['relative_date_days'].dt.days
            df['relative_date_weeks'] = df['relative_date_days']/7
            df['relative_date_months'] = df['relative_date_days']/30.5
            df['relative_date_years'] = df['relative_date_days']/365

            df['Technology'] = key
        else:
            df_temp = dfs[key][1].copy(deep=True)

            df_temp['relative_date_days'] = df_temp['Announced Date'] - (df_return[(df_return['Key']==key) & (df_return['Title']=='Earliest Moment of Peak Hype')]['Value']).iloc[0]
            df_temp['relative_date_days'] = df_temp['relative_date_days'].dt.days
            df_temp['relative_date_weeks'] = df_temp['relative_date_days']/7
            df_temp['relative_date_months'] = df_temp['relative_date_days']/30.5
            df_temp['relative_date_years'] = df_temp['relative_date_days']/365

            df_temp['Technology'] = key            
                        
            df = df.append(df_temp, ignore_index=True)
        
        count += 1

        # NEED TO CONVERT EACH $ TO A DATAPOINT
        
    return df